In [1]:
import numpy as np

# Binomial Tree Model for European and American Options
def binomial_tree(S, K, T, r, sigma, N, option_type="call", is_american=False):
    """
    S: Current stock price
    K: Strike price
    T: Time to expiration (in years)
    r: Risk-free interest rate
    sigma: Volatility of the underlying asset
    N: Number of time steps in the binomial tree
    option_type: "call" or "put"
    is_american: If True, allow early exercise (American option); else, European option
    """
    
    # Calculate parameters
    dt = T / N  # Time step
    u = np.exp(sigma * np.sqrt(dt))  # Up movement factor
    d = 1 / u  # Down movement factor
    p = (np.exp(r * dt) - d) / (u - d)  # Risk-neutral probability of up movement
    
    # Initialize stock prices at maturity
    stock_prices = np.zeros(N + 1)
    option_values = np.zeros(N + 1)
    
    # Stock prices at expiration
    for i in range(N + 1):
        stock_prices[i] = S * (u ** i) * (d ** (N - i))
    
    # Option values at expiration
    if option_type == "call":
        option_values = np.maximum(stock_prices - K, 0)
    elif option_type == "put":
        option_values = np.maximum(K - stock_prices, 0)
    
    # Backward induction through the tree
    for j in range(N - 1, -1, -1):
        for i in range(j + 1):
            option_values[i] = np.exp(-r * dt) * (p * option_values[i + 1] + (1 - p) * option_values[i])
            
            if is_american:
                stock_price = S * (u ** i) * (d ** (j - i))
                if option_type == "call":
                    option_values[i] = np.maximum(option_values[i], stock_price - K)
                elif option_type == "put":
                    option_values[i] = np.maximum(option_values[i], K - stock_price)
    
    return option_values[0]

# Parameters
S = 100   # Current stock price
K = 100   # Strike price
T = 1     # Time to expiration (in years)
r = 0.05  # Risk-free interest rate
sigma = 0.2  # Volatility (20%)
N = 100   # Number of time steps

# Calculate the option prices for European and American options
european_call_price = binomial_tree(S, K, T, r, sigma, N, option_type="call", is_american=False)
european_put_price = binomial_tree(S, K, T, r, sigma, N, option_type="put", is_american=False)

american_call_price = binomial_tree(S, K, T, r, sigma, N, option_type="call", is_american=True)
american_put_price = binomial_tree(S, K, T, r, sigma, N, option_type="put", is_american=True)

print(f"European Call Option Price: {european_call_price:.2f}")
print(f"European Put Option Price: {european_put_price:.2f}")
print(f"American Call Option Price: {american_call_price:.2f}")
print(f"American Put Option Price: {american_put_price:.2f}")


European Call Option Price: 10.43
European Put Option Price: 5.55
American Call Option Price: 10.43
American Put Option Price: 6.08
